In [ ]:
input = read("day6.txt", String)


input_rows = split(input, "\n")
row_length = length(input_rows[1]) + 1
col_length = length(input_rows)

chart = [((mod(i, row_length), col_length - floor(Int, i/row_length)), char) for (i, char) in enumerate(input)]
const chart_size =  [row_length-1,col_length]

const blockages = [x for (x, y) in chart if y == '#']
guard_location = [x for (x, y) in chart if y == '^'][1]
guard_dir = "N"

#println(blockages)

In [ ]:
function update_guard_location(start_loc, start_dir, blockages, chart_size)
    stop = 0
    if start_dir == "N"
        blockages_ahead = [(x,y) for (x,y) in blockages if (x == start_loc[1]) & (y > start_loc[2])] 
        if isempty(blockages_ahead)
            stop = 1
            new_loc = (start_loc[1], chart_size[2])
            new_dir = "N"
        else
            sort!(blockages_ahead, by = x -> x[2])
            blockage_hit = blockages_ahead[1]
            new_loc = (blockage_hit[1], blockage_hit[2]-1)
            new_dir = "E"
        end
        visited_locs = [(start_loc[1], i) for i in start_loc[2]:new_loc[2]]
    elseif start_dir == "S"
        blockages_ahead = [(x,y) for (x,y) in blockages if (x == start_loc[1]) & (y < start_loc[2])] 
        if isempty(blockages_ahead)
            stop = 1
            new_loc = (start_loc[1], 1)
            new_dir = "S"
        else
            sort!(blockages_ahead, by = x -> x[2], rev = true)
            blockage_hit = blockages_ahead[1]
            new_loc = (blockage_hit[1], blockage_hit[2]+1)
            new_dir = "W"
        end
        visited_locs = [(start_loc[1], i) for i in new_loc[2]:start_loc[2]]
    elseif start_dir == "E"
        blockages_ahead = [(x,y) for (x,y) in blockages if (x > start_loc[1]) & (y == start_loc[2])] 
        if isempty(blockages_ahead)
            stop = 1
            new_loc = (chart_size[1], start_loc[2])
            new_dir = "E"
        else
            sort!(blockages_ahead, by = x -> x[1])
            blockage_hit = blockages_ahead[1]
            new_loc = (blockage_hit[1]-1, blockage_hit[2])
            new_dir = "S"
        end
        visited_locs = [(i, start_loc[2]) for i in start_loc[1]:new_loc[1]]
    elseif start_dir == "W"
        blockages_ahead = [(x,y) for (x,y) in blockages if (x < start_loc[1]) & (y == start_loc[2])] 
        if isempty(blockages_ahead)
            stop = 1
            new_loc = (0, start_loc[2])
            new_dir = "W"
        else
            sort!(blockages_ahead, by = x -> x[1], rev = true)
            blockage_hit = blockages_ahead[1]
            new_loc = (blockage_hit[1]+1, blockage_hit[2])
            new_dir = "N"
        end
        visited_locs = [(i, start_loc[2]) for i in new_loc[1]:start_loc[1]]
    end
    
    visited_locs_and_dirs = [(l, start_dir) for l in visited_locs if l != new_loc]
    return(new_loc, new_dir, stop, visited_locs, visited_locs_and_dirs)
end

function guard_travel(guard_location, guard_dir, blockages, chart_size)
    stop = 0
    visited_locs_set = Set()
    while stop == 0
        #println(guard_location, guard_dir)
        guard_location, guard_dir, stop, visited_locs, _ = update_guard_location(guard_location, guard_dir, blockages, chart_size)
        union!(visited_locs_set, visited_locs)
    end
    return(visited_locs_set)
end

@time visited_locs_set = guard_travel(guard_location, guard_dir, blockages, chart_size)
@time visited_locs_set = guard_travel(guard_location, guard_dir, blockages, chart_size)

#println(guard_location, guard_dir)
println("Puzzle 1: ", length(visited_locs_set))

In [ ]:

function is_guard_travel_loop(guard_location, guard_dir, blockages, chart_size)
    stop = 0
    visited_locs_and_dirs_set = Set()
    while stop == 0
        #println(guard_location, guard_dir)
        guard_location, guard_dir, stop, x, visited_locs_and_dirs = update_guard_location(guard_location, guard_dir, blockages, chart_size)
        union!(visited_locs_and_dirs_set, visited_locs_and_dirs)
        if (guard_location, guard_dir) in visited_locs_and_dirs_set
            return(true)
        end
    end
    return(false)
end

# free_space = [x for (x, y) in chart if y == '.']
# actually don't need to test all free space, just the original path solution
blockages_test_list = [vcat(blockages, l) for l in visited_locs_set]

@time loop_bool = [is_guard_travel_loop(guard_location, guard_dir, test_blockages, chart_size) for test_blockages in blockages_test_list]

println("Puzzle 2: ", sum(loop_bool))